<p>
  <a href="https://colab.research.google.com/github/neo4j-partners/hands-on-lab-neo4j-and-vertex-ai/blob/main/Lab%203%20-%20Knowledge%20Graph%20Creation%20with%20Generative%20AI/exploring-pandas.ipynb" target="_blank">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
  </a>
</p>

We're going to play around with some of our structured data. Let’s get it from cloud storage.

In [ ]:
!wget https://storage.googleapis.com/neo4j-datasets/form13/form13-v2.csv

Now let's load our data into memory and get a peek at it.

In [ ]:
import pandas as pd
df = pd.read_csv('form13-v2.csv')
df

Each row of the data represents common stock a manager holds within a given report period.  The relevant columns are 
- `ManagerName`: Name of filing manager for the form13 - the entity owning the stock. We will consider `managerName` a unique id for our intents and purposes. 
- `reportCalendarOrQuarter`: The report calendar year of quarter
- `companyName`: Name of the stock issuing company
- `cusip`: Unique 9-digit identifier for the stock
- `value`: Market value of the stock
- `shares`: number of shares


Let's do a few aggregations to understand the data better.

## Aggregations

In [ ]:
# number of managers
unique_managers = df.managerName.unique().size
print(f'Number of unique managers: {unique_managers:,}')

In [ ]:
# number of companies (use cusip)
unique_companies = df.cusip.unique().size
print(f'Number of unique companys: {unique_companies:,}')

In [ ]:
# number of holdings per quarter
df[['reportCalendarOrQuarter', 'cusip']].groupby('reportCalendarOrQuarter').count().rename(columns={'cusip':'holdingCount'})

In [ ]:
# distribution of holdings by managers

manager_count_df = (df[['managerName', 'cusip']]
                    .groupby(['managerName'])
                    .count()
                    .rename(columns={'cusip':'holdingCount'}))
manager_count_df.hist(bins=50)

In [ ]:
# distribution of holdings by managers

print(f'--- Manager Holding Distribution Stats ---')

print(f'Min: {manager_count_df.holdingCount.min():,}')
print(f'Mean: {round(manager_count_df.holdingCount.mean(), 2):,}')
print(f'95th percentile: {round(manager_count_df.holdingCount.quantile(q=0.95),2):,}')
print(f'99th percentile: {round(manager_count_df.holdingCount.quantile(q=0.99),2):,}')
print(f'Max: {manager_count_df.holdingCount.max():,}')

You will notice that we have a skewed distribution for managers.  Most managers own relatively few assets, while a small subset owns a lot.  This imbalance will be important to remember for later labs as we analyze our data inside graph. 

You will also notice below that the managers who own the most overtime tend to be large institutions. 

In [ ]:
# Who are the managers that have had the most holdings overtime

manager_count_df.sort_values(by='holdingCount', ascending=False).head(10)